## Toronto Neighborhood Exploration

In [102]:
# Install geopy for plotting
!pip install geopy
!conda install -c conda-forge folium=0.5.0 --yes

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Defaulting to user installation because normal site-packages is not writeable
Solving environment: done

# All requested packages already installed.



In [92]:
#Import database
import pandas as pd
import requests
import numpy as np
import json
from geopy.geocoders import nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
import lxml.html
from pandas import DataFrame
print('libarary imported')

libarary imported


In [93]:
# Read data into dataframe
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)
Table=tables[0]
Table

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [94]:
# Remove rows with Not assigned Borough
Table=Table[Table['Borough'] !='Not assigned']
Table

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [95]:
# Check whether Neighbourhood has Not assigned item
Table[Table['Neighbourhood']=='Not assigned'].count()

Postal Code      0
Borough          0
Neighbourhood    0
dtype: int64

In [96]:
# Combine postal code where multiple neighbourhoods exist
Table.groupby('Postal Code').Neighbourhood.agg([('Borough', ','.join), ('Neighbourhood', ', '.join)])

,Borough,Neighbourhood
Postal Code,,
M1B,"Malvern, Rouge","Malvern, Rouge"
M1C,"Rouge Hill, Port Union, Highland Creek","Rouge Hill, Port Union, Highland Creek"
M1E,"Guildwood, Morningside, West Hill","Guildwood, Morningside, West Hill"
M1G,Woburn,Woburn
M1H,Cedarbrae,Cedarbrae
...,...,...
M9N,Weston,Weston
M9P,Westmount,Westmount
M9R,"Kingsview Village, St. Phillips, Martin Grove ...","Kingsview Village, St. Phillips, Martin Grove ..."


In [108]:
Table=Table.sort_values(by='Postal Code')

In [109]:
# Reset index
Table_index=Table.reset_index(drop=True)
Table_index

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [98]:
Table_index.shape

(103, 3)

In [110]:
url2='http://cocl.us/Geospatial_data'
index = pd.read_csv(url2)

In [111]:
index

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [114]:
Table_index[['Latitude','Longitude']]=index[['Latitude','Longitude']]

In [116]:
Tab=Table_index
Tab

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [130]:
# Create dataframe where Borough contains 'Toronto'
Tab=Tab[Tab["Borough"].str.contains("Toronto", na=False)]
Tab

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [136]:
#create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[Tab.iloc[0,3], Tab.iloc[0,4]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Tab['Latitude'], Tab['Longitude'], Tab['Borough'], Tab['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto